In [129]:
import dask.dataframe as dd

In [130]:
df = dd.read_csv('City_Sponsored_Events_Simple.csv')

In [131]:
df.head()

,Event,Start time,End time,Event Description/Agenda,Street Address,Latitude,Longitude,Location,Event Info Url,Classification,Audience
0,"Northwest Flower & Garden Show, Feb. 17-21, 2016",02/18/2016 12:00:00 AM,02/19/2016 12:00:00 AM,"Northwest Flower & Garden Show, Feb. 17-21, 2016",800 Convention Pl,47.61195,-122.33167,"(47.61195, -122.33167)",http://www.seattle.gov/living-in-seattle/event...,"Community,Education,Festivals/Fairs,Gardening,...",All
1,Society of Women Engineers ABJ Regional Confer...,02/18/2016 12:00:00 AM,02/19/2016 12:00:00 AM,Society of Women Engineers ABJ Regional Confer...,2211 Alaskan Way,47.61137,-122.34947,"(47.61137, -122.34947)",http://www.seattle.gov/living-in-seattle/event...,"Special Events,Training-Classes/Workshops","Adults,Children"
2,Design Commission Meeting,02/18/2016 08:30:00 AM,02/18/2016 04:30:00 PM,Design Commission Meeting,Boards and Commissions Room,NaN,NaN,NaN,http://www.seattle.gov/living-in-seattle/event...,Boards & Commissions,Adults
3,Design Commission Meeting - Seattle Opera,02/18/2016 09:00:00 AM,02/18/2016 11:00:00 AM,Design Commission Meeting - Seattle Opera,Boards and Commissions Room,NaN,NaN,NaN,http://www.seattle.gov/living-in-seattle/event...,Boards & Commissions,Adults
4,"CANCELED - Parks, Seattle Center, Libraries & ...",02/18/2016 09:30:00 AM,02/18/2016 09:30:00 AM,"CANCELED - Parks, Seattle Center, Libraries & ...",NaN,NaN,NaN,NaN,http://www.seattle.gov/living-in-seattle/event...,City Council Meeting,"Adults,All"


In [132]:
del df['Street Address']
del df['Event Info Url']

In [133]:
df.head()

,Event,Start time,End time,Event Description/Agenda,Latitude,Longitude,Location,Classification,Audience
0,"Northwest Flower & Garden Show, Feb. 17-21, 2016",02/18/2016 12:00:00 AM,02/19/2016 12:00:00 AM,"Northwest Flower & Garden Show, Feb. 17-21, 2016",47.61195,-122.33167,"(47.61195, -122.33167)","Community,Education,Festivals/Fairs,Gardening,...",All
1,Society of Women Engineers ABJ Regional Confer...,02/18/2016 12:00:00 AM,02/19/2016 12:00:00 AM,Society of Women Engineers ABJ Regional Confer...,47.61137,-122.34947,"(47.61137, -122.34947)","Special Events,Training-Classes/Workshops","Adults,Children"
2,Design Commission Meeting,02/18/2016 08:30:00 AM,02/18/2016 04:30:00 PM,Design Commission Meeting,NaN,NaN,NaN,Boards & Commissions,Adults
3,Design Commission Meeting - Seattle Opera,02/18/2016 09:00:00 AM,02/18/2016 11:00:00 AM,Design Commission Meeting - Seattle Opera,NaN,NaN,NaN,Boards & Commissions,Adults
4,"CANCELED - Parks, Seattle Center, Libraries & ...",02/18/2016 09:30:00 AM,02/18/2016 09:30:00 AM,"CANCELED - Parks, Seattle Center, Libraries & ...",NaN,NaN,NaN,City Council Meeting,"Adults,All"


In [134]:
def extractDate(df, col):
    df[col] = dd.to_datetime(df[col])
    df['Year'] = df[col].dt.year
    df['Month'] = df[col].dt.month
    df['Day'] = df[col].dt.day

In [135]:
extractDate(df, 'Start time')

In [136]:
def filterMonths(month):
    if (month == 12) or (month == 1) or (month == 2):
        season = 'WINTER'
    elif (month == 3) or (month == 4) or (month == 5):
        season = 'SPRING'
    elif(month == 6) or (month == 7) or (month == 8):
        season = 'SUMMER'
    else:
        season = 'FALL'
    return season

def createSeasonColumn(df):
    df['Season'] = df.Month.apply(filterMonths)

In [137]:
createSeasonColumn(df)

C:\Users\Sebastian\AppData\Local\conda\conda\envs\spark\lib\site-packages\dask\dataframe\core.py:2252: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [138]:
import pandas as pd

In [139]:
pddf = df.compute()

In [140]:
del df

In [141]:
def encodeCat(df, col):
    dfCat = df[col].str.get_dummies(sep=',')
    new_df = pd.concat([df, dfCat], sort=False, axis=1)
    return new_df

In [142]:
pddf = encodeCat(pddf, 'Classification')
pddf = encodeCat(pddf, 'Audience')
pddf = encodeCat(pddf, 'Season')

In [143]:
pddf.head()

,Event,Start time,End time,Event Description/Agenda,Latitude,Longitude,Location,Classification,Audience,Year,...,Adults,All,Children,Family,Pets,Senior,Special Needs,Teen,SPRING,WINTER
0,"Northwest Flower & Garden Show, Feb. 17-21, 2016",2016-02-18 00:00:00,02/19/2016 12:00:00 AM,"Northwest Flower & Garden Show, Feb. 17-21, 2016",47.61195,-122.33167,"(47.61195, -122.33167)","Community,Education,Festivals/Fairs,Gardening,...",All,2016,...,0,1,0,0,0,0,0,0,0,1
1,Society of Women Engineers ABJ Regional Confer...,2016-02-18 00:00:00,02/19/2016 12:00:00 AM,Society of Women Engineers ABJ Regional Confer...,47.61137,-122.34947,"(47.61137, -122.34947)","Special Events,Training-Classes/Workshops","Adults,Children",2016,...,1,0,1,0,0,0,0,0,0,1
2,Design Commission Meeting,2016-02-18 08:30:00,02/18/2016 04:30:00 PM,Design Commission Meeting,NaN,NaN,NaN,Boards & Commissions,Adults,2016,...,1,0,0,0,0,0,0,0,0,1
3,Design Commission Meeting - Seattle Opera,2016-02-18 09:00:00,02/18/2016 11:00:00 AM,Design Commission Meeting - Seattle Opera,NaN,NaN,NaN,Boards & Commissions,Adults,2016,...,1,0,0,0,0,0,0,0,0,1
4,"CANCELED - Parks, Seattle Center, Libraries & ...",2016-02-18 09:30:00,02/18/2016 09:30:00 AM,"CANCELED - Parks, Seattle Center, Libraries & ...",NaN,NaN,NaN,City Council Meeting,"Adults,All",2016,...,1,1,0,0,0,0,0,0,0,1


In [144]:
pddf.head()

,Event,Start time,End time,Event Description/Agenda,Latitude,Longitude,Location,Classification,Audience,Year,...,Adults,All,Children,Family,Pets,Senior,Special Needs,Teen,SPRING,WINTER
0,"Northwest Flower & Garden Show, Feb. 17-21, 2016",2016-02-18 00:00:00,02/19/2016 12:00:00 AM,"Northwest Flower & Garden Show, Feb. 17-21, 2016",47.61195,-122.33167,"(47.61195, -122.33167)","Community,Education,Festivals/Fairs,Gardening,...",All,2016,...,0,1,0,0,0,0,0,0,0,1
1,Society of Women Engineers ABJ Regional Confer...,2016-02-18 00:00:00,02/19/2016 12:00:00 AM,Society of Women Engineers ABJ Regional Confer...,47.61137,-122.34947,"(47.61137, -122.34947)","Special Events,Training-Classes/Workshops","Adults,Children",2016,...,1,0,1,0,0,0,0,0,0,1
2,Design Commission Meeting,2016-02-18 08:30:00,02/18/2016 04:30:00 PM,Design Commission Meeting,NaN,NaN,NaN,Boards & Commissions,Adults,2016,...,1,0,0,0,0,0,0,0,0,1
3,Design Commission Meeting - Seattle Opera,2016-02-18 09:00:00,02/18/2016 11:00:00 AM,Design Commission Meeting - Seattle Opera,NaN,NaN,NaN,Boards & Commissions,Adults,2016,...,1,0,0,0,0,0,0,0,0,1
4,"CANCELED - Parks, Seattle Center, Libraries & ...",2016-02-18 09:30:00,02/18/2016 09:30:00 AM,"CANCELED - Parks, Seattle Center, Libraries & ...",NaN,NaN,NaN,City Council Meeting,"Adults,All",2016,...,1,1,0,0,0,0,0,0,0,1


In [145]:
pddf.columns

Index(['Event', 'Start time', 'End time', 'Event Description/Agenda',
       'Latitude', 'Longitude', 'Location', 'Classification', 'Audience',
       'Year', 'Month', 'Day', 'Season', 'Arts', 'Boards & Commissions',
       'Business/Trade', 'City Council Meeting', 'Community', 'Dance',
       'Education', 'Ethnic/Cultural', 'Festivals/Fairs', 'Film', 'Gardening',
       'Grants', 'Hobbies', 'Holiday/Seasonal Events', 'LGBT',
       'Museum/Gallery', 'Music', 'Nature/Environment',
       'Neighborhood Meetings', 'Other', 'Public Forum',
       'Public Outreach and Engagement', 'Public Safety', 'Special Events',
       'Sports/Recreation', 'Technology', 'Theater', 'Tourism',
       'Training-Classes/Workshops', 'Transportation', 'Volunteer/Work Party',
       'Youth', 'Adults', 'All', 'Children', 'Family', 'Pets', 'Senior',
       'Special Needs', 'Teen', 'SPRING', 'WINTER'],
      dtype='object')